# Notebook to download all reference papers and extract txt files from them
I've made a list of references - "reference_list.txt" and reading information from there. 

In [2]:
reference_txt = open("reference_list.txt", 'r').readlines()

In [3]:
# Get titles from 
def parse_reference(ref):
    asd = ",".join(ref.split(",")[:-1])
    asd = asd.split(". ")[-1]
    # asd = list(filter(lambda x: len(x) > 10, asd))
    return asd

list_of_paper_names = []

for r in reference_txt:
    list_of_paper_names.append(parse_reference(r))
    

In [4]:
list_of_paper_names

['Alloying behavior of iron, gold and silver in AlCoCrCuNi-based equimolar high-entropy alloys',
 'FCC and BCC equivalents in as-cast solid solutions of AlxCo yCrzCu0.5FevNiw high-entropy alloys',
 'Alloying behavior in multi-component AlCoCrCuFe and NiCoCrCuFe high entropy alloys',
 'Effects of Mn, Ti and V on the microstructure and properties of AlCrFeCoNiCu high entropy alloy',
 'Microstructure and compressive properties of multicomponent Alx(TiVCrMnFeCoNiCu)100−x high-entropy alloys',
 'Microstructures and compressive properties of multicomponent AlCoCrCuFeNiMox alloys',
 'Phase transformation induced by lattice distortion in multiprincipal component CoCrFeNiCuxAl1−x solid-solution alloys',
 'Morphology, structure and composition of precipitates in Al0.3CoCrCu0.5FeNi high-entropy alloy',
 'Nanostructured high-entropy alloys with multiple principal elements: Novel alloy design concepts and outcomes',
 'Effect of the substitution of Co by Mn in Al-Cr-Cu-Fe-Co-Ni high-entropy alloys',

In [5]:
from scidownl import scihub_download
import os
import re

try:
    last_pdf_done = max(list(map(lambda x: int(re.search("reference_([0-9]+)", x).groups()[0]), os.listdir("references/"))))
except: 
    last_pdf_done = 0

print(f"Starting from reference_{last_pdf_done+1}")
# Needs to be re-run if stuck
for i in range(len(list_of_paper_names)):
    if i + 1 <= last_pdf_done or i+1 in [41]: # 41 gets stuck, idk
        continue
    print(i+1)
    scihub_download(list_of_paper_names[i], paper_type="title", out=f"references/reference_{i+1}.pdf")

[INFO] | 2023/12/06 17:14:57 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2023/12/06 17:14:57 | <- Request: scihub_url=https://sci-hub.ru, source=TitleSource[type=title, id=Phase equilibria in the quaternary molybdenum-ruthenium-rhodium-palladium system], proxies={}


Starting from reference_121
121


[INFO] | 2023/12/06 17:14:58 | -> Response: status_code=200, content_length=5888
[WARNING] | 2023/12/06 17:14:58 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/12/06 17:14:58 | Choose scihub url [1]: http://sci-hub.ru
[INFO] | 2023/12/06 17:14:58 | <- Request: scihub_url=http://sci-hub.ru, source=TitleSource[type=title, id=Phase equilibria in the quaternary molybdenum-ruthenium-rhodium-palladium system], proxies={}
[INFO] | 2023/12/06 17:14:59 | -> Response: status_code=200, content_length=5888
[WARNING] | 2023/12/06 17:14:59 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/12/06 17:14:59 | Choose scihub url [2]: https://sci-hub.st
[INFO] | 2023/12/06 17:14:59 | <- Request: scihub_url=https://sci-hub.st, source=TitleSource[type=title, id=Phase equilibria in the quaternary molybdenum-ruthenium-rhodium-palladium syst

Lets see which ones are missing

In [6]:
not_dones = sorted(list(set(list(range(len(list_of_paper_names)))) - set(list(map(lambda x: int(re.search("reference_([0-9]+)", x).groups()[0]), os.listdir("references/"))))))
print(f"Missing references: {len(not_dones)}/{len(list_of_paper_names)} (can't do anything about it..)")
", ".join(list(map(str, not_dones)))

Missing references: 41/121 (can't do anything about it..)


'0, 2, 8, 10, 13, 15, 16, 18, 19, 20, 21, 22, 23, 32, 34, 39, 40, 41, 42, 49, 51, 61, 62, 65, 66, 67, 71, 74, 75, 77, 78, 80, 83, 84, 85, 94, 97, 98, 108, 110, 115'

# Convert to txt files (along with some processing to only keep "content")

In [7]:
from pdfminer.high_level import extract_text
import glob
import tqdm

In [24]:
def print_lines_removed(lx1, lx2):
    print(f"Lines removed: {len(lx1)} - {len(lx2)} = {len(lx1) - len(lx2)}\n", "\n".join(set(lx1)- set(lx2)))
    print("#"*200)

def add_sentence_to_paragraph(p, s):
    if len(p) == 0:
        return s

    # If '-' at end, remove and join
    if p[-1] == '-':
        return p[:-1] + s

    # If '' at end, then add ' ' and join
    else:
        return p + ' ' + s

def process_raw_pdf_text(raw, debug=False):
    l1 = list(filter(lambda x: len(x) > 1, raw.split("\n")))
    
    # Get rid of everything before abstract
    sec_l = ["Abstract", "Introduction"]
    for i in range(len(l1)):
        # if l1[i].startswith("Abstract") or l1[i].startswith("ABSTRACT"):
        if any([x in l1[i] for x in (sec_l + list(map(lambda y: y.upper(), sec_l)))]):
            l2 = l1[i:]
            break
    
    if debug:
        print_lines_removed(l1, l2)
    
    # Remove lines with some fixed text # Should be special (if small, then should have special char), otherwise it may be subtrings eslewhere
    unwanted_text = ["All rights reserved", "E-mail address:", "Tel.:", "fax:", "doi:", "Keywords:", "\\"]
    
    l3 = []
    for line in l2:
        if not any([x in line for x in unwanted_text]):
            l3.append(line)
            # Stop before acknowledgement or references
            if line.endswith("Acknowledgement") or line.endswith("ACKNOWLEDGEMENT") or \
                line.endswith("Acknowledgment") or line.endswith("ACKNOWLEDGMENT") or \
                line.endswith("References") or line.endswith("REFERENCES"):
                l3.pop(-1)
                break
    
    if debug:
        print_lines_removed(l2, l3)
    
    # Generate paragraphs - Logic: Paragraphs sentences are almost equal in length
    s_len = 0
    l4 = []
    

    
    cur_para = ''
    start = 0
    l3 += ''
    for i in range(len(l3)-1):
        # Continuing the para
        cur_para = add_sentence_to_paragraph(cur_para, l3[i])
    
        # Ending the para (small line, or ending in '.') - Captures most cases. Any other are too annoying to bother with.
        # Better to stop early and divide a single para into multiple ones, than have some unwanted text added to it
        if cur_para[-1] == '.' or len(l3[i]) < len(l3[i+1]) * 0.8: # Using a 0.8 factor
            if debug:
                print(l3[start:i+1])
                print()
                print(cur_para)
                print("-" * 150)
                start = i + 1
            l4.append(cur_para)
            cur_para = ''
    if debug:
        print("#" * 200)
    
    # Some postprocessings
    # Should contain any common english words like "the", "is" or section heading
    sec_heads = ["Abstract", "Introduction", "Experiment", "Results", "Discussion", "Method"]
    common_eng_words = [" the ", " is ", " with ", " of ", " be ", " to ", " a ", " in ", " and ", " have ", " it ", " for ", " on ", " not ", " did ", " we ", " an ",
                       "The ", "It ", "A ", 
                       ]
    wanted = common_eng_words + sec_heads + list(map(lambda x: x.upper(), sec_heads))
    l5 = list(filter( lambda line: any([x in line for x in wanted]), l4))
    if debug:
        print_lines_removed(l4, l5)

    # Now lets deal with figure and table captions
    # First, we separate them
    l6 = []
    l7 = []
    for line in l5:
        if line.startswith("Fig") or line.startswith("Table"):
            l7.append(line)
        else:
            l6.append(line)

    # Now combine paragraphs if they are "incomplete"
    l8 = [l6[0]]
    for line in l6[1:]:
        if line[0].islower() and l8[-1][-1] != ".":
            l8[-1] = l8[-1]+line
        else:
            l8.append(line)

    # Finally, put back the Fig and Table lines (hopefully) where they belong
    for i in range(len(l5)):
        if l5[i].startswith("Fig") or l5[i].startswith("Table"):
            # Find where the previous line went and fit in this line there
            for j in range(len(l8)):
                if l5[i-1] in l8[j]:
                    l8 = l8[:j+1] + [l5[i]] + l8[j+1:]
                    break
    
    return "\n".join(l8)

The ones with  
local variable 'l2' referenced before assignment  
error don't have "abstract" or "intro" section.. Idk what to do about them so let's ignore them! :)

The others errors are invalid pdfs

In [44]:
for file in glob.glob("references/*"):
    try: # Some still give error - Can't be bothered
        raw = extract_text(file)
        open(file.replace("pdf", "txt").replace("references", "reference_txts"), 'w').write(process_raw_pdf_text(raw, debug=False))
        print(file, "DONE!")
    except Exception as e:
        print(file, f": {e}")

# raw = extract_text("references/reference_1.pdf")

references/reference_86.pdf DONE!
references/reference_106.pdf DONE!
references/reference_103.pdf DONE!
references/reference_45.pdf DONE!
references/reference_118.pdf : local variable 'l2' referenced before assignment
references/reference_91.pdf DONE!
references/reference_95.pdf DONE!
references/reference_57.pdf DONE!
references/reference_64.pdf DONE!
references/reference_87.pdf DONE!
references/reference_120.pdf : No /Root object! - Is this really a PDF?
references/reference_73.pdf DONE!
references/reference_3.pdf DONE!
references/reference_36.pdf DONE!
references/reference_29.pdf DONE!
references/reference_119.pdf DONE!
references/reference_88.pdf : No /Root object! - Is this really a PDF?
references/reference_114.pdf DONE!
references/reference_14.pdf DONE!
references/reference_43.pdf DONE!
references/reference_4.pdf DONE!
references/reference_72.pdf DONE!
references/reference_11.pdf DONE!
references/reference_24.pdf : local variable 'l2' referenced before assignment
references/refer

In [7]:
# Some files are still problematic
import glob
import os
for file in sorted(glob.glob("reference_txts/*")):
    if len(open(file, 'r').read()) < 5000:
        os.remove(file)
        print(f"Removing {file}")